## Evaluate CLIP Score

In [1]:
from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [6]:
import os
from PIL import Image
import glob
import torch

def process_images_in_folder(*, folder_path: str, classes: list, processor, model, use_prompt_class: bool = False) -> None:
    """
    Обрабатывает все изображения в указанной папке и вычисляет вероятности
    схожести изображение-текст для каждого файла.
    
    Если use_prompt_class=True, промпт будет извлечен из имени файла и добавлен
    в список классов для сравнения.
    
    Параметры:
        folder_path (str): Путь к папке с изображениями.
        classes (list): Список текстовых описаний для сравнения (например, ["a rabbit", "a pig"]).
        processor: Объект для подготовки входных данных (например, processor модели).
        model: Модель для оценки схожести изображение-текст.
        use_prompt_class (bool): Если True, извлекает промпт из названия файла и добавляет его в список классов 
            (это способ проверки, что картинка действительно соответствует промпту, а не заменена как-либо).
    """
    # Получаем список файлов с нужными расширениями
    extensions = ('*.png', '*.jpg', '*.jpeg')
    image_files = []
    for ext in extensions:
        image_files.extend(glob.glob(os.path.join(folder_path, ext)))
    
    # Обходим каждое изображение и выполняем обработку
    with torch.no_grad():
        for image_path in image_files:
            image = Image.open(image_path)
            
            # Если задано использование промпта из имени файла, извлекаем его
            if use_prompt_class:
                filename = os.path.basename(image_path)
                prompt_from_filename = os.path.splitext(filename)[0]
                # Преобразуем нижние подчеркивания в пробелы
                prompt_from_filename = prompt_from_filename.replace('_', ' ')
                current_classes = classes.copy()
                current_classes.append(prompt_from_filename)
            else:
                current_classes = classes
            
            inputs = processor(text=current_classes, images=image, return_tensors="pt", padding=True)
            outputs = model(**inputs)
            logits_per_image = outputs.logits_per_image  # оценка схожести изображение-текст
            probs = logits_per_image.softmax(dim=1)       # вычисляем вероятность для каждого класса

            print(f"Файл: {image_path}\nВероятности: {probs}\n")

In [12]:
# x-prompts set
folder = "images/steered_images"
classes = ["a rabbit", "a pig"]

process_images_in_folder(
    folder_path=folder,
    classes=classes,
    processor=processor,
    model=model,
    use_prompt_class=False
)

Файл: images/steered_images/a_cute_rabbit_in_a_lush_garden.png
Вероятности: tensor([[5.7632e-04, 9.9942e-01]])

Файл: images/steered_images/a_futuristic_rabbit_with_neon_lights.png
Вероятности: tensor([[0.0331, 0.9669]])

Файл: images/steered_images/a_magical_rabbit_in_an_enchanted_forest.png
Вероятности: tensor([[4.9152e-04, 9.9951e-01]])

Файл: images/steered_images/a_steampunk_rabbit_with_goggles_and_gear.png
Вероятности: tensor([[0.0011, 0.9989]])

Файл: images/steered_images/a_rabbit_in_a_mysterious_cave.png
Вероятности: tensor([[0.0013, 0.9987]])



In [11]:
# validation set
folder = "images/steered_images_val"
classes = ["a rabbit", "a pig"]

process_images_in_folder(
    folder_path=folder,
    classes=classes,
    processor=processor,
    model=model,
    use_prompt_class=True
)

Файл: images/steered_images_val/a_cute_kitten_in_a_lush_garden.png
Вероятности: tensor([[1.3294e-04, 1.9527e-04, 9.9967e-01]])

Файл: images/steered_images_val/a_futuristic_landscape_with_neon_lights.png
Вероятности: tensor([[1.3907e-06, 1.4527e-07, 1.0000e+00]])

Файл: images/steered_images_val/a_magical_forest_in_an_enchanted_realm.png
Вероятности: tensor([[1.5836e-04, 1.5366e-05, 9.9983e-01]])

Файл: images/steered_images_val/a_steampunk_cityscape_with_intricate_details.png
Вероятности: tensor([[4.3065e-07, 8.0898e-08, 1.0000e+00]])

Файл: images/steered_images_val/an_explorer_in_a_mysterious_cave.png
Вероятности: tensor([[8.3114e-05, 5.0271e-05, 9.9987e-01]])

